In [3]:
import pandas as pd
import sys
import matplotlib.pylab as plt
import operator

In [4]:
uid_songid_file_location = './Dataset/Result_by_userid_count_multiple_songs.csv'
songid_uid_file_location = './Dataset/Result_by_songid_count_multiple_users.csv'
songid_mappings_file_location = './Dataset/kaggle_songs.txt'
uid_songid_triplets_file_location = './Dataset/kaggle_visible_evaluation_triplets.txt'

In [10]:
uid_songid_file = open(uid_songid_file_location)
uid_songid_file_data = []
for line in uid_songid_file:
    to_be_appended = line.strip().replace('[','').replace(']','').replace('"','').replace("'",'').strip().replace(' ','').split(',')
    uid_songid_file_data.append(to_be_appended[1:])
uid_songid_file_data = uid_songid_file_data[1:]
uid_songid_file.close()

In [11]:
songid_mappings_file = open(songid_mappings_file_location)
songid_mappings = {}
for line in songid_mappings_file:
    #print(line)
    temp_var = line.strip().split()
    songid_mappings[temp_var[0]] = int(temp_var[1])

songid_mappings_file.close()
#songid_mappings

In [12]:
#In transactional type data, convert hashed song_id to its corresponding numerical song_id 

for itemset in uid_songid_file_data:
    for item_num in range(len(itemset)):
        if(type(itemset[item_num]) == str):
            itemset[item_num] = itemset[item_num].strip()
            itemset[item_num] = songid_mappings[itemset[item_num]]   
    itemset.sort()

In [14]:
uid_songid_file_data[13]

[5001, 64151, 125810, 125810, 202552, 251217, 282660, 298741]

In [15]:
#Returns the global frequency of every song.
#A dictionary whose key is the song_id(in numerical format) and its value is the total 
#number of times it was heard by all the users.

def get_global_song_frequency(songid_mappings, song_count_filename):
    
    frequency_dictionary ={}
    song_count_file = open(song_count_filename)
    for data in song_count_file:
        temp_var = data.strip().split('\t')[1:]
        mapped_index = songid_mappings[temp_var[0]]
        if mapped_index in frequency_dictionary:
            frequency_dictionary[mapped_index] += int(temp_var[1])
        else:
            frequency_dictionary[mapped_index] = int(temp_var[1])
            
    return frequency_dictionary

fd = get_global_song_frequency(songid_mappings,uid_songid_triplets_file_location)

In [16]:
def get_songs_with_freq_10_to_5000(all_song_frequency_distribution_dict):
    
    filtered_song_list = []
    for song_id,frequency in all_song_frequency_distribution_dict.items():
        if(frequency>10 and frequency<=5000):
            filtered_song_list.append(song_id)
    
    filtered_song_list.sort()
    return filtered_song_list

filtered_song_list = get_songs_with_freq_10_to_5000(fd)

In [19]:
# Remove those songs from the transaction that are not frequent.

for i in range(len(uid_songid_file_data)):
    uid_songid_file_data[i].sort()
    itemset = uid_songid_file_data[i].copy()
    for item_num in uid_songid_file_data[i]:
        if item_num not in filtered_song_list:
            #itemset = filter(lambda a: a != item_num, itemset)
            #itemset.remove(item_num)
            while item_num in itemset: itemset.remove(item_num)
    uid_songid_file_data[i] = itemset
            

In [25]:
uid_songid_file_data[3]

[41446,
 83123,
 85724,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 115697,
 126580,
 126580,
 126580,
 141927,
 168365,
 228083,
 289108,
 289108,
 289108,
 289108,
 375283]

In [21]:
#Write the reduced dataset(frequency of songs in [11,5000]) in a file 

with open('./Dataset/filtered_uid_songid_dataset_freq_11_to_5000_with_repetitions.txt', 'w') as filehandle:
    for itemset in uid_songid_file_data:
        filehandle.write('%s\n' % itemset)

In [26]:
#Modify the data to reflect if a song is preferred with high, medium or low frequency by a user. 
#NOTE: Call this only once. Calling it again and again can append L/M/H again and again and we don't want that.

for user_num in range(len(uid_songid_file_data)):
    itemset = uid_songid_file_data[user_num]
    sid_dict = {}
    new_itemset = []
    for item in itemset:
        if(item in sid_dict):
            sid_dict[item] += 1
        else:
            sid_dict[item] = 1
    for sid,frequency in sid_dict.items():
        new_sid = str(sid)
        if(frequency<5):
            new_sid += 'L'
        if(frequency>=5 and frequency<10):
            new_sid += 'M'
        if(frequency>=10):
            new_sid +='H'
        new_itemset.append(new_sid)
        
    uid_songid_file_data[user_num] = new_itemset

uid_songid_file_data[3]

['41446L',
 '83123L',
 '85724L',
 '115697H',
 '126580L',
 '141927L',
 '168365L',
 '228083L',
 '289108L',
 '375283L']

In [27]:
#Write the reduced dataset(frequency of songs in [11,5000]) in a file 

with open('./Dataset/filtered_uid_songid_dataset_freq_11_to_5000_with_HML.txt', 'w') as filehandle:
    for itemset in uid_songid_file_data:
        filehandle.write('%s\n' % itemset)